<a href="https://colab.research.google.com/github/ha-seungwon/Dacon/blob/main/dna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [404]:

#%% [code]
import pandas as pd
import random
import os
import numpy as np
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score, f1_score
class CFG:
    SEED = 42


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정


#%% [code]
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')


#%% [code]
train=train.drop(['father','mother','gender'],axis=1)
test=test.drop(['father','mother','gender'],axis=1)


def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x


#%% [code]
train_x, train_y = get_x_y(train)
test_x = get_x_y(test)






In [405]:
info=pd.read_csv('snp_info.csv')

In [406]:
'''train_x['SNP_02'].value_counts()

for i in train:
  plt.title(i)
  #plt.plot(train[train['class']=='A'][i])
  plt.plot(train[train['class']=='B'][i])
  plt.plot(train[train['class']=='C'][i])
  plt.legend(['B','C'])
  plt.show()


'''

"train_x['SNP_02'].value_counts()\n\nfor i in train:\n  plt.title(i)\n  #plt.plot(train[train['class']=='A'][i])\n  plt.plot(train[train['class']=='B'][i])\n  plt.plot(train[train['class']=='C'][i])\n  plt.legend(['B','C'])\n  plt.show()\n\n\n"

In [407]:
train = pd.read_csv('./train.csv')
train['gender'].unique()

array([0])

In [408]:
def dna_percnt(df):
  a=[]
  c=[]
  g=[]
  t=[]
  for i in df.index:
      string=df.iloc[i]['SNP_01':'SNP_15']
      string=''.join(string)
      string=string.replace(" ", "")
      if 'T' in string:
        print('tttt')
      a.append(string.count('A'))
      c.append(string.count('C'))
      g.append(string.count('G'))
  df['A']=a
  df['G']=g
  df['C']=c
  df['A_percent']=df['A']/30
  df['G_percent']=df['G']/30
  df['C_percent']=df['C']/30
  return df
train_x=dna_percnt(train_x)
test_x=dna_percnt(test_x)

In [409]:
train_x
def seperate_snp(df):
  for i in df:
    if 'SNP' in i:
      temp=df[i]
      left_temp=[]
      right_temp=[]
      for index,value in enumerate(temp):
        temp_date=value.split(' ')
        left_temp.append(temp_date[0])
        right_temp.append(temp_date[1])
      df[i+'left']=left_temp
      df[i+'right']=right_temp
  return df

train_x=seperate_snp(train_x)
test_x=seperate_snp(test_x)

In [410]:
class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]


In [411]:
'''
for i in range(len(snp_col)):
  snp_col.append(snp_col[i]+'left')
  snp_col.append(snp_col[i]+'right')'''

"\nfor i in range(len(snp_col)):\n  snp_col.append(snp_col[i]+'left')\n  snp_col.append(snp_col[i]+'right')"

In [412]:

#%% [code]
snp_data = []
for col in snp_col:
    snp_data += list(train_x[col].values)

#코드 셀 <undefined>
#%% [code]
train_y = class_le.fit_transform(train_y)
snp_le.fit(snp_data)



for col in train_x.columns:
    if col in snp_col:
        train_x[col] = snp_le.transform(train_x[col])
        test_x[col] = snp_le.transform(test_x[col])


snp_col.append('trait')
for i in train_x.columns:
  if i in snp_col:
    train_x=pd.get_dummies(train_x,columns=[i])


for i in test_x.columns:
  if i in snp_col:
    test_x=pd.get_dummies(test_x,columns=[i])


#train_x=pd.get_dummies(train_x,columns=['trait'])
#test_x=pd.get_dummies(test_x,columns=['trait'])


In [413]:
t=[0,1,2]

train_x = train_x.replace('A', 0)
train_x = train_x.replace('G', 1)
train_x = train_x.replace('C', 2)

test_x = test_x.replace('A', 0)
test_x = test_x.replace('G', 1)
test_x = test_x.replace('C', 2)


In [414]:
train_x

,A,G,C,A_percent,G_percent,C_percent,SNP_01left,SNP_01right,SNP_02left,SNP_02right,...,SNP_12_5,SNP_13_0,SNP_13_1,SNP_13_5,SNP_14_0,SNP_14_2,SNP_14_3,SNP_15_0,SNP_15_4,SNP_15_5
0,20,9,1,0.666667,0.300000,0.033333,1,1,0,1,...,0,1,0,0,1,0,0,1,0,0
1,21,8,1,0.700000,0.266667,0.033333,0,1,0,1,...,0,0,0,1,1,0,0,1,0,0
2,18,10,2,0.600000,0.333333,0.066667,1,1,1,1,...,0,1,0,0,1,0,0,1,0,0
3,12,18,0,0.400000,0.600000,0.000000,0,0,1,1,...,1,0,0,1,1,0,0,0,0,1
4,18,8,4,0.600000,0.266667,0.133333,1,1,1,1,...,0,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,19,9,2,0.633333,0.300000,0.066667,0,1,0,1,...,0,1,0,0,1,0,0,1,0,0
258,20,9,1,0.666667,0.300000,0.033333,1,1,0,0,...,0,0,1,0,1,0,0,0,1,0
259,12,17,1,0.400000,0.566667,0.033333,0,1,1,1,...,1,0,0,1,0,1,0,0,0,1
260,14,15,1,0.466667,0.500000,0.033333,0,0,1,1,...,0,0,0,1,0,1,0,0,0,1


In [415]:
'''grap=train_x
grap['t']=train_y
plt.figure(figsize=(150,150))
sns.heatmap(data = grap.corr(), annot=True, 
fmt = '.2f', linewidths=.5, cmap='Blues')
'''

"grap=train_x\ngrap['t']=train_y\nplt.figure(figsize=(150,150))\nsns.heatmap(data = grap.corr(), annot=True, \nfmt = '.2f', linewidths=.5, cmap='Blues')\n"

In [416]:

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split


from sklearn.model_selection import KFold


In [417]:
from sklearn.model_selection import GridSearchCV
# XGB
param_xgb = {"max_depth": [10,30,50],
              "min_child_weight" : [1,3,6,10],
              "n_estimators": [10,50,100,200]}
xgb = XGBClassifier()
             
      

In [418]:
import optuna
import sklearn
import psutil
import time
X_train, X_test, y_train, y_test = train_test_split(train_x,train_y,test_size=0.3, random_state=42,shuffle=False)

gscv_xgb = GridSearchCV (estimator = xgb, param_grid = param_xgb, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
gscv_xgb.fit(train_x, train_y)

Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] END ..max_depth=10, min_child_weight=1, n_estimators=10; total time=   0.0s
[CV] END ..max_depth=10, min_child_weight=1, n_estimators=10; total time=   0.0s
[CV] END ..max_depth=10, min_child_weight=1, n_estimators=10; total time=   0.0s
[CV] END ..max_depth=10, min_child_weight=1, n_estimators=50; total time=   0.1s
[CV] END ..max_depth=10, min_child_weight=1, n_estimators=50; total time=   0.1s
[CV] END ..max_depth=10, min_child_weight=1, n_estimators=50; total time=   0.1s
[CV] END .max_depth=10, min_child_weight=1, n_estimators=100; total time=   0.2s
[CV] END .max_depth=10, min_child_weight=1, n_estimators=100; total time=   0.2s
[CV] END .max_depth=10, min_child_weight=1, n_estimators=100; total time=   0.2s
[CV] END .max_depth=10, min_child_weight=1, n_estimators=200; total time=   0.3s
[CV] END .max_depth=10, min_child_weight=1, n_estimators=200; total time=   0.3s
[CV] END .max_depth=10, min_child_weight=1, n_e

GridSearchCV(cv=3, estimator=XGBClassifier(), n_jobs=1,
             param_grid={'max_depth': [10, 30, 50],
                         'min_child_weight': [1, 3, 6, 10],
                         'n_estimators': [10, 50, 100, 200]},
             scoring='accuracy', verbose=2)

In [419]:
print('XGB 파라미터: ', gscv_xgb.best_params_)
print('XGB 예측 정확도: {:.4f}'.format(gscv_xgb.best_score_))

XGB 파라미터:  {'max_depth': 10, 'min_child_weight': 6, 'n_estimators': 100}
XGB 예측 정확도: 0.9351


In [421]:

m1=RandomForestClassifier()
m2=XGBClassifier(**gscv_xgb.best_params_)
m3=CatBoostClassifier(objective= "MultiClass",
    task_type= "CPU")
m4=lgb.LGBMClassifier( random_state = 42)

mlp_model = MLPClassifier(hidden_layer_sizes=(200,200,100), max_iter=300,activation = 'relu',solver='adam',random_state=42)

result=[]

m1.fit(X_train,y_train)
result.append(accuracy_score(m1.predict(X_test),y_test))

m2.fit(X_train,y_train)
result.append(accuracy_score(m2.predict(X_test),y_test))

m3.fit(X_train,y_train)
result.append(accuracy_score(m3.predict(X_test),y_test))


m4.fit(X_train,y_train)
result.append(accuracy_score(m4.predict(X_test),y_test))


#vt_clf.fit(X_train,y_train)
#result.append(accuracy_score(vt_clf.predict(X_test),y_test))
model= KNeighborsClassifier(n_neighbors=3)
model.fit(X_train,y_train)
result.append(accuracy_score(model.predict(X_test),y_test))


mlp_model.fit(X_train,y_train)
result.append(accuracy_score(mlp_model.predict(X_test),y_test))

Learning rate set to 0.072562
0:	learn: 1.0360775	total: 11.1ms	remaining: 11.1s
1:	learn: 0.9865870	total: 32.3ms	remaining: 16.1s
2:	learn: 0.9360984	total: 51.1ms	remaining: 17s
3:	learn: 0.8962074	total: 71.1ms	remaining: 17.7s
4:	learn: 0.8556319	total: 90.5ms	remaining: 18s
5:	learn: 0.8234454	total: 110ms	remaining: 18.2s
6:	learn: 0.7820829	total: 132ms	remaining: 18.7s
7:	learn: 0.7440201	total: 142ms	remaining: 17.6s
8:	learn: 0.7139129	total: 163ms	remaining: 18s
9:	learn: 0.6870248	total: 182ms	remaining: 18s
10:	learn: 0.6593458	total: 204ms	remaining: 18.4s
11:	learn: 0.6358459	total: 222ms	remaining: 18.3s
12:	learn: 0.6131882	total: 247ms	remaining: 18.7s
13:	learn: 0.5919807	total: 266ms	remaining: 18.7s
14:	learn: 0.5723375	total: 284ms	remaining: 18.6s
15:	learn: 0.5532855	total: 305ms	remaining: 18.8s
16:	learn: 0.5324124	total: 321ms	remaining: 18.6s
17:	learn: 0.5181841	total: 340ms	remaining: 18.5s
18:	learn: 0.5004090	total: 358ms	remaining: 18.5s
19:	learn: 0.4

In [422]:

models=['RandomForestClassifier','XGBClassifier','CatBoostClassifier','LGBMClassifier','knn','MLPClassifier']

for index,value in enumerate(models):
  print(value,':',result[index])
#[0.9312977099236641,

RandomForestClassifier : 0.9493670886075949
XGBClassifier : 0.8987341772151899
CatBoostClassifier : 0.9240506329113924
LGBMClassifier : 0.9493670886075949
knn : 0.9240506329113924
MLPClassifier : 0.9746835443037974


In [381]:
train_x

,A,G,C,A_percent,G_percent,C_percent,SNP_01left,SNP_01right,SNP_02left,SNP_02right,...,SNP_13_0,SNP_13_1,SNP_13_5,SNP_14_0,SNP_14_2,SNP_14_3,SNP_15_0,SNP_15_4,SNP_15_5,t
0,20,9,1,0.666667,0.300000,0.033333,1,1,0,1,...,1,0,0,1,0,0,1,0,0,1
1,21,8,1,0.700000,0.266667,0.033333,0,1,0,1,...,0,0,1,1,0,0,1,0,0,2
2,18,10,2,0.600000,0.333333,0.066667,1,1,1,1,...,1,0,0,1,0,0,1,0,0,1
3,12,18,0,0.400000,0.600000,0.000000,0,0,1,1,...,0,0,1,1,0,0,0,0,1,0
4,18,8,4,0.600000,0.266667,0.133333,1,1,1,1,...,0,1,0,1,0,0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,19,9,2,0.633333,0.300000,0.066667,0,1,0,1,...,1,0,0,1,0,0,1,0,0,1
258,20,9,1,0.666667,0.300000,0.033333,1,1,0,0,...,0,1,0,1,0,0,0,1,0,2
259,12,17,1,0.400000,0.566667,0.033333,0,1,1,1,...,0,0,1,0,1,0,0,0,1,0
260,14,15,1,0.466667,0.500000,0.033333,0,0,1,1,...,0,0,1,0,1,0,0,0,1,0


In [423]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import time
import warnings
from sklearn.model_selection import RepeatedStratifiedKFold
warnings.filterwarnings('ignore')
catb_params = {
    "objective": "MultiClass",
    "task_type": "CPU",
}
RANDOM_STATE = 12 
catb_predictions = []
catb_scores = []
X=train_x.to_numpy()
y=train_y
best_model = None
best_score=-1
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
#kf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
for fold, (train_idx, valid_idx) in enumerate(kf.split(X = X, y = y)):
    print(10*"=", f"Fold={fold+1}", 10*"=")
    
    start_time = time.time()
    x_train = X[train_idx, :]
    x_valid = X[valid_idx, :]
    y_train = y[train_idx]
    y_valid = y[valid_idx]
    
    model = CatBoostClassifier(**catb_params)
    #model=XGBClassifier()
    model.fit(x_train, y_train,
          early_stopping_rounds=200,
          eval_set=[(x_valid, y_valid)],
          verbose=0)
    
    preds_valid = model.predict(x_valid)
    acc = accuracy_score(y_valid,  preds_valid)
    catb_scores.append(acc)
    run_time = time.time() - start_time
    if best_score <acc:
      best_score=acc
      best_model=model
    print(f"Fold={fold+1}, acc: {acc:.8f}, Run Time: {run_time:.2f}")
   
    test_preds = model.predict(test_x)
    catb_predictions.append(test_preds)
    
print("Mean Accuracy:", np.mean(catb_scores))

========== Fold=1 ==========
Fold=1, acc: 0.90566038, Run Time: 6.43
========== Fold=2 ==========
Fold=2, acc: 0.94339623, Run Time: 15.73
========== Fold=3 ==========
Fold=3, acc: 0.92307692, Run Time: 10.04
========== Fold=4 ==========
Fold=4, acc: 0.96153846, Run Time: 13.10
========== Fold=5 ==========
Fold=5, acc: 0.94230769, Run Time: 5.25
Mean Accuracy: 0.9351959361393325


In [424]:
train_x

,A,G,C,A_percent,G_percent,C_percent,SNP_01left,SNP_01right,SNP_02left,SNP_02right,...,SNP_12_5,SNP_13_0,SNP_13_1,SNP_13_5,SNP_14_0,SNP_14_2,SNP_14_3,SNP_15_0,SNP_15_4,SNP_15_5
0,20,9,1,0.666667,0.300000,0.033333,1,1,0,1,...,0,1,0,0,1,0,0,1,0,0
1,21,8,1,0.700000,0.266667,0.033333,0,1,0,1,...,0,0,0,1,1,0,0,1,0,0
2,18,10,2,0.600000,0.333333,0.066667,1,1,1,1,...,0,1,0,0,1,0,0,1,0,0
3,12,18,0,0.400000,0.600000,0.000000,0,0,1,1,...,1,0,0,1,1,0,0,0,0,1
4,18,8,4,0.600000,0.266667,0.133333,1,1,1,1,...,0,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,19,9,2,0.633333,0.300000,0.066667,0,1,0,1,...,0,1,0,0,1,0,0,1,0,0
258,20,9,1,0.666667,0.300000,0.033333,1,1,0,0,...,0,0,1,0,1,0,0,0,1,0
259,12,17,1,0.400000,0.566667,0.033333,0,1,1,1,...,1,0,0,1,0,1,0,0,0,1
260,14,15,1,0.466667,0.500000,0.033333,0,0,1,1,...,0,0,0,1,0,1,0,0,0,1


In [426]:
mlp_model = MLPClassifier(hidden_layer_sizes=(200,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=42)
mlp_model.fit(train_x,train_y)

xg_model=XGBClassifier(max_depth=10, min_child_weight=1, n_estimators= 10)
xg_model.fit(train_x,train_y)

lgb_clf = lgb.LGBMClassifier( random_state = 42)
lgb_clf.fit(train_x,train_y)

from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(train_x,train_y)

#clf = GradientBoostingClassifier(n_estimators=50, learning_rate=1.0,max_depth=1, random_state=0).fit(X_train, y_train)


knn_model= KNeighborsClassifier(n_neighbors=3)
knn_model.fit(train_x,train_y)

KNeighborsClassifier(n_neighbors=3)

In [427]:
from sklearn.ensemble import VotingClassifier,VotingClassifier,GradientBoostingClassifier
vt_clf = VotingClassifier(estimators = [
    ('rf', rf),('xgb', xg_model),('lgb', lgb_clf),('mlp',mlp_model),

], voting = 'soft')




In [428]:
train_x

,A,G,C,A_percent,G_percent,C_percent,SNP_01left,SNP_01right,SNP_02left,SNP_02right,...,SNP_12_5,SNP_13_0,SNP_13_1,SNP_13_5,SNP_14_0,SNP_14_2,SNP_14_3,SNP_15_0,SNP_15_4,SNP_15_5
0,20,9,1,0.666667,0.300000,0.033333,1,1,0,1,...,0,1,0,0,1,0,0,1,0,0
1,21,8,1,0.700000,0.266667,0.033333,0,1,0,1,...,0,0,0,1,1,0,0,1,0,0
2,18,10,2,0.600000,0.333333,0.066667,1,1,1,1,...,0,1,0,0,1,0,0,1,0,0
3,12,18,0,0.400000,0.600000,0.000000,0,0,1,1,...,1,0,0,1,1,0,0,0,0,1
4,18,8,4,0.600000,0.266667,0.133333,1,1,1,1,...,0,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,19,9,2,0.633333,0.300000,0.066667,0,1,0,1,...,0,1,0,0,1,0,0,1,0,0
258,20,9,1,0.666667,0.300000,0.033333,1,1,0,0,...,0,0,1,0,1,0,0,0,1,0
259,12,17,1,0.400000,0.566667,0.033333,0,1,1,1,...,1,0,0,1,0,1,0,0,0,1
260,14,15,1,0.466667,0.500000,0.033333,0,0,1,1,...,0,0,0,1,0,1,0,0,0,1


In [429]:
vt_clf.fit(train_x,train_y)
voting_pred=vt_clf.predict(test_x)

xg_model.fit(train_x,train_y)
xg_pred=xg_model.predict(test_x)

mlp_model.fit(train_x,train_y)
mlp_pred=mlp_model.predict(test_x)

In [430]:
voting_pred

A_CLASS=[  0,   4,   8,   9,  13,  16,  23,  27,  28,  30,  31,  34,  40,
             50,  52,  57,  60,  65,  67,  68,  72,  73,  82,  89,  96,  98,
            100, 104, 105, 108, 115, 116, 120, 128, 130, 131, 136, 137, 139,
            140, 141, 146, 149, 150, 152, 155, 156, 159, 164, 165, 169
]

#B 는 SNP_14 A A




for i in A_CLASS:
  if submit.iloc[i]['class'] != 'A':
    print(i)
    print('여기틀림')


In [431]:
B=test[test['SNP_14']=='A A'].index.tolist()

In [439]:
#%% [code]
submit = pd.read_csv('./sample_submission.csv')


#%% [code]
submit['class'] = class_le.inverse_transform(xg_pred)


#%% [code]
submit.to_csv('./submit.csv', index=False)




In [440]:
submit['class'].value_counts()

#'''B    85
#A    50
#C    40
#mlp

#B    87
#A    51
#C    37
#xgb

#B    85
#A    51
#C    39
#voting_pred

B    87
A    51
C    37
Name: class, dtype: int64

In [ ]:
#A 51 이게 맞음

In [ ]:
'''

0초
submit['class'].value_counts()
B    86
A    51
C    38
Name: class, dtype: int64

B    85
A    51
C    39
Name: class, dtype: int64

Fold=3, acc: 0.94230769, Run Time: 2.88
0.9719171917	
'''

'''B    86
A    51
C    38
Name: class, dtype: int64

Fold=2, acc: 0.98113208, Run Time: 5.37
0.9622367466	
========== Fold=3 =========='''

'B    86\nA    51\nC    38\nName: class, dtype: int64\n\nFold=2, acc: 0.98113208, Run Time: 5.37\n0.9622367466\t\n========== Fold=3 =========='

0.8987341772151899
0.9113924050632911
Learning rate set to 0.072562
0:	learn: 1.0370658	total: 8.23ms	remaining: 8.22s
1:	learn: 0.9774129	total: 15ms	remaining: 7.51s
2:	learn: 0.9319054	total: 22.1ms	remaining: 7.34s
3:	learn: 0.8909648	total: 29ms	remaining: 7.22s
4:	learn: 0.8464256	total: 35.8ms	remaining: 7.12s
5:	learn: 0.8159023	total: 42.6ms	remaining: 7.06s
6:	learn: 0.7812531	total: 49.5ms	remaining: 7.02s
7:	learn: 0.7462517	total: 56.2ms	remaining: 6.96s
8:	learn: 0.7153750	total: 63.1ms	remaining: 6.95s
9:	learn: 0.6831764	total: 69.7ms	remaining: 6.9s
10:	learn: 0.6591508	total: 76.5ms	remaining: 6.88s
11:	learn: 0.6369116	total: 88.3ms	remaining: 7.27s
12:	learn: 0.6130292	total: 95.3ms	remaining: 7.23s
13:	learn: 0.5898161	total: 102ms	remaining: 7.21s
14:	learn: 0.5708521	total: 109ms	remaining: 7.17s
15:	learn: 0.5480169	total: 116ms	remaining: 7.15s
16:	learn: 0.5312844	total: 125ms	remaining: 7.21s
17:	learn: 0.5157168	total: 131ms	remaining: 7.17s
18:	learn: 0.499

In [ ]:
for i in range(len(result)):
  result[i]=''.join(result[i])


#Codon Table Key는 DNA codon이며, value는 Amino acid 실제 값입니다.
table = {
'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L', 'TCT': 'S',
'TCC': 'S', 'TCA': 'S', 'TCG': 'S', 'TAT': 'Y', 'TAC': 'Y',
'TGT': 'C', 'TGC': 'C', 'TGG': 'W', 'CTT': 'L', 'CTC': 'L',
'CTA': 'L', 'CTG': 'L', 'CCT': 'P', 'CCC': 'P', 'CCA': 'P',
'CCG': 'P', 'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R', 'ATT': 'I',
'ATC': 'I', 'ATA': 'I', 'ATG': 'M', 'ACT': 'T', 'ACC': 'T',
'ACA': 'T', 'ACG': 'T', 'AAT': 'N', 'AAC': 'N', 'AAA': 'K',
'AAG': 'K', 'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V', 'GCT': 'A',
'GCC': 'A', 'GCA': 'A', 'GCG': 'A', 'GAT': 'D', 'GAC': 'D',
'GAA': 'E', 'GAG': 'E', 'GGT': 'G', 'GGC': 'G', 'GGA': 'G',
'GGG': 'G'}
# Extra data in case you want it.
stop_codons = [ 'TAA', 'TAG', 'TGA']

result=[]
if __name__ == "__main__":
    for i in stop_codons:
        table[i] = '*' # table에서 stop codon 자체를 '*'로 바꿔줍니다.
    for i in test_x.index:
      string=test_x.iloc[i]['SNP_01':'SNP_15']
      string=''.join(string)
      string=string.replace(" ", "")
      seq = string
      protein = [] # protein이 저장될 list입니다. append 되며 점점 converting 될것입니다.
      ch_finish = False # stop codon이 나왔느냐 안나왔느냐에 대한 check 루틴입니다.

      for codon in range(0,len(seq)-len(seq)%3, 3): # 3개씩 이어나갈 코돈에서 혹시 남을 1개 혹은 2개의 nucleotide에 대해 처리해줍니다.
          if ch_finish==False: # Stop codon이 나오지 않은 경우 protein으로 계속 번역합니다.
              protein.append(table[seq[codon:codon+3]])
              if table[seq[codon:codon+3]] == '*':
                  ch_finish = True
          else: # stop codon이 한번 나오면 나머지 codon은 모두 * 처리합니다.
              protein.append('*')
      result.append(protein)
    print("".join(protein)) # list 형태의 amino acid 변수를 구분자 없이 연결해주면서 출력합니다.

for i in range(len(result)):
  result[i]=''.join(result[i])

#test_x['codon']=result
#train_x['codon']=result


#Codon Table Key는 DNA codon이며, value는 Amino acid 실제 값입니다.
table = {
'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L', 'TCT': 'S',
'TCC': 'S', 'TCA': 'S', 'TCG': 'S', 'TAT': 'Y', 'TAC': 'Y',
'TGT': 'C', 'TGC': 'C', 'TGG': 'W', 'CTT': 'L', 'CTC': 'L',
'CTA': 'L', 'CTG': 'L', 'CCT': 'P', 'CCC': 'P', 'CCA': 'P',
'CCG': 'P', 'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R', 'ATT': 'I',
'ATC': 'I', 'ATA': 'I', 'ATG': 'M', 'ACT': 'T', 'ACC': 'T',
'ACA': 'T', 'ACG': 'T', 'AAT': 'N', 'AAC': 'N', 'AAA': 'K',
'AAG': 'K', 'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V', 'GCT': 'A',
'GCC': 'A', 'GCA': 'A', 'GCG': 'A', 'GAT': 'D', 'GAC': 'D',
'GAA': 'E', 'GAG': 'E', 'GGT': 'G', 'GGC': 'G', 'GGA': 'G',
'GGG': 'G'}
# Extra data in case you want it.
stop_codons = [ 'TAA', 'TAG', 'TGA']

result=[]
if __name__ == "__main__":
    for i in stop_codons:
        table[i] = '*' # table에서 stop codon 자체를 '*'로 바꿔줍니다.
    for i in test_x.index:
      string=test_x.iloc[i]['SNP_01':'SNP_15']
      string=''.join(string)
      string=string.replace(" ", "")
      seq = string
      protein = [] # protein이 저장될 list입니다. append 되며 점점 converting 될것입니다.
      ch_finish = False # stop codon이 나왔느냐 안나왔느냐에 대한 check 루틴입니다.

      for codon in range(0,len(seq)-len(seq)%3, 3): # 3개씩 이어나갈 코돈에서 혹시 남을 1개 혹은 2개의 nucleotide에 대해 처리해줍니다.
          if ch_finish==False: # Stop codon이 나오지 않은 경우 protein으로 계속 번역합니다.
              protein.append(table[seq[codon:codon+3]])
              if table[seq[codon:codon+3]] == '*':
                  ch_finish = True
          else: # stop codon이 한번 나오면 나머지 codon은 모두 * 처리합니다.
              protein.append('*')
      result.append(protein)
    print("".join(protein)) # list 형태의 amino acid 변수를 구분자 없이 연결해주면서 출력합니다.

for i in range(len(result)):
  result[i]=''.join(result[i])

#test_x['codon']=result